In [1]:
import argparse
import torch
import torch.utils.data
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from pydoc import locate
import tensorboardX
from tqdm import tqdm
import os, shutil
from torchsummary import summary

from utils import * 
from models import *

In [2]:
import pandas as pd
import numpy as np
import gc
from tqdm import tqdm_notebook
import copy

import transforms3d
import open3d as o3d

In [3]:
# DYNAMIC_TRAIN_FOLDER_PATH = '/home/sabyasachi/Projects/ati/data/data/datasets/Carla/small_map/110k/dynamic_training_data/npy_data'
DYNAMIC_PREPROCESS_DATA_PATH = "../training_data/dynamic_only/data"
DYNAMIC_PREPROCESS_GT_PATH = "../training_data/dynamic_only/label"

In [8]:
def getint(name):
    try:
        return int(name.split('.')[0])
    except Exception as e:
        print("Error occured while trying to read {}".format(name))
        return None

In [9]:
# train_file  = sorted(os.listdir( DYNAMIC_TRAIN_FOLDER_PATH), key=getint)[0]
val_file  = sorted(os.listdir( DYNAMIC_PREPROCESS_DATA_PATH), key=getint)[-1]
npyList = sorted(os.listdir(DYNAMIC_PREPROCESS_DATA_PATH), key=getint)[:-1]

val_gt_file  = sorted(os.listdir( DYNAMIC_PREPROCESS_GT_PATH), key=getint)[-1]
npy_gt_List = sorted(os.listdir(DYNAMIC_PREPROCESS_GT_PATH), key=getint)[:-1]

In [18]:
print("Loading dynamic training data and labels")
dynamic_dataset_train_dict = {}
for data_file, gt_file in tqdm_notebook(zip(npyList, npy_gt_List), total=len(npyList)):
    some_dynamic_dataset = {}
    some_dynamic_dataset['data'] = np.load(os.path.join(DYNAMIC_PREPROCESS_DATA_PATH, data_file), allow_pickle=True)
    some_dynamic_dataset['label'] = pd.read_csv(os.path.join(DYNAMIC_PREPROCESS_GT_PATH, gt_file))
    dynamic_dataset_train_dict[getint(data_file)] = some_dynamic_dataset
print("Loading dynamic validation data and labels")
dynamic_dataset_val = {}
dynamic_dataset_val['data'] = np.load(os.path.join(DYNAMIC_PREPROCESS_DATA_PATH, val_file), allow_pickle=True)
dynamic_dataset_val['label'] = pd.read_csv(os.path.join(DYNAMIC_PREPROCESS_GT_PATH, val_gt_file))
print("done")

Loading dynamic training data and labels



Loading dynamic validation data and labels
done


In [29]:
dynamic_dataset_train_dict[1]['label'].iloc[1]

step                          2.000000e+00
platform_timestamp            1.282469e+09
platform_timestamp_convert    6.412345e+10
game_timestamp                2.000000e+02
game_timestamp_convert        1.010000e+04
location_x                    2.377000e+02
location_x_convert            1.546000e+03
location_y                    1.297500e+02
location_y_convert            8.890000e+02
location_z                    3.937349e+01
orientation_x                -1.000000e+00
orientation_x_convert         9.300000e+01
orientation_y                 4.053116e-06
orientation_y_convert         1.000000e+02
orientation_z                 0.000000e+00
rotation_pitch                0.000000e+00
rotation_pitch_convert        1.000000e+02
rotation_yaw                  1.799998e+02
rotation_yaw_convert          1.195000e+03
rotation_roll                 0.000000e+00
acceleration_x               -1.824564e-08
acceleration_x_convert        1.000000e+02
acceleration_y                7.395171e-14
acceleratio

In [35]:
for key, value in dynamic_dataset_train_dict.items():
    print(key)
    print(value['label'])

1
      step  platform_timestamp  platform_timestamp_convert  game_timestamp  \
0        1          1282468750                6.412344e+10             100   
1        2          1282469000                6.412345e+10             200   
2        3          1282469125                6.412346e+10             300   
3        4          1282469375                6.412347e+10             400   
4        5          1282469500                6.412348e+10             500   
5        6          1282469625                6.412348e+10             600   
6        7          1282469750                6.412349e+10             700   
7        8          1282469875                6.412349e+10             800   
8        9          1282470000                6.412350e+10             900   
9       10          1282470125                6.412351e+10            1000   
10      11          1282470250                6.412351e+10            1100   
11      12          1282470375                6.412352e+10    

In [37]:
dynamic_dataset_train_dict[1]['data'].shape

(1345, 2, 32, 512)

In [40]:
dynamic_dataset_train_dict[1]['label'].iloc[0].to_dict()

{'step': 1.0,
 'platform_timestamp': 1282468750.0,
 'platform_timestamp_convert': 64123437600.0,
 'game_timestamp': 100.0,
 'game_timestamp_convert': 5100.0,
 'location_x': 237.6999969482422,
 'location_x_convert': 1546.0,
 'location_y': 129.75,
 'location_y_convert': 889.0,
 'location_z': 39.430625915527344,
 'orientation_x': -1.0,
 'orientation_x_convert': 93.0,
 'orientation_y': 4.0531158447265625e-06,
 'orientation_y_convert': 100.0,
 'orientation_z': 0.0,
 'rotation_pitch': 0.0,
 'rotation_pitch_convert': 100.0,
 'rotation_yaw': 179.999755859375,
 'rotation_yaw_convert': 1195.0,
 'rotation_roll': 0.0,
 'acceleration_x': 0.0,
 'acceleration_x_convert': 100.0,
 'acceleration_y': 0.0,
 'acceleration_y_convert': 100.0,
 'acceleration_z': 0.0,
 'forward_speed': 0.0,
 'steer': 3.2697405458748108e-06,
 'throttle': 1.0}

In [ ]:
def pose2matrix(translation_list, rotation_angle_list, zoom_list=[1,1,1]):
    trans_vec = np.array(translation_list)
    rot_ang = [np.deg2rad(ang) for ang in rotation_angle_list ]
    rot_mat = transforms3d.euler.euler2mat(rot_ang[0], rot_ang[1], rot_ang[2])
    zoom_vec = np.array(zoom_list)
    transform_mat = transforms3d.affines.compose(trans_vec, rot_mat, zoom_vec)
    return transform_mat

def draw_registration_result(src, dst, transformation):
    source = copy.deepcopy(src)
    target = copy.deepcopy(dst)
    
    source.paint_uniform_color([1, 0, 0]) # red
    target.paint_uniform_color([0, 0, 1]) # blue
    source.transform(transformation)
    o3d.visualization.draw_geometries([source, target], width=1280, height=800)

def slam_loss_fn(prev_pcd, next_pcd, idx, VOXEL_SZ=0.2):
    def transform_lidar_to_gt_frame(src, dst):
        source = copy.deepcopy(src)
        target = copy.deepcopy(dst)
        transformation_lidar2gt = pose2matrix([0,0,0], [0,0,90],[1,1,-1])
        source.transform(transformation_lidar2gt)
        target.transform(transformation_lidar2gt)
        return source, target
    
    def get_gt_pose(df_gt, idx):
        # get_gt_pose execution starts here
        prev_gt = df_gt[df_gt['step'] == idx +1].iloc[0]
        next_gt = df_gt[df_gt['step'] == idx+1 +1].iloc[0]

        prev_inv_mat = np.linalg.inv(pose2matrix([prev_gt['location_x'],
                                                  prev_gt['location_y'],
                                                  prev_gt['location_z']],
                                                 [prev_gt['rotation_roll'],
                                                  prev_gt['rotation_pitch'],
                                                  prev_gt['rotation_yaw']]))
        next_mat = pose2matrix([next_gt['location_x'],
                                next_gt['location_y'],
                                next_gt['location_z']],
                               [next_gt['rotation_roll'],
                                next_gt['rotation_pitch'],
                                next_gt['rotation_yaw']])
        transformation_gt = np.matmul(prev_inv_mat, next_mat)
        transformation_gt = np.linalg.inv(transformation_gt) # Open 3d assumes transform is applied on source and not target
        return transformation_gt
    
    def get_icp_pose(src, dst):
        def crop_pcd(old_pcd, crop_min_arr=np.array([-100,-100,-2]), crop_max_arr=np.array([100,100,100])):
            np.random.seed(0)
            pcd = copy.deepcopy(old_pcd)

            cropped_pcd = o3d.geometry.crop_point_cloud(pcd, crop_min_arr, crop_max_arr)
            pcd = cropped_pcd
            return pcd

        def prepare_dataset(source, target, voxel_size):
            def preprocess_point_cloud(pcd, voxel_size):
                pcd_down = o3d.geometry.voxel_down_sample(pcd, voxel_size)
                radius_normal = voxel_size * 2
                o3d.geometry.estimate_normals(
                    pcd_down,
                    o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
                radius_feature = voxel_size * 5
                pcd_fpfh = o3d.registration.compute_fpfh_feature(
                    pcd_down,
                    o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
                return pcd_down, pcd_fpfh
            source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
            target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
            return source_down, target_down, source_fpfh, target_fpfh

        def execute_global_registration(source_down, target_down, source_fpfh,
                                        target_fpfh, voxel_size):
            distance_threshold = voxel_size * 1.5
            result = o3d.registration.registration_ransac_based_on_feature_matching(
                source_down, target_down, source_fpfh, target_fpfh, distance_threshold,
                o3d.registration.TransformationEstimationPointToPoint(False), 4, [
                    o3d.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
                    o3d.registration.CorrespondenceCheckerBasedOnDistance(
                        distance_threshold)
                ], o3d.registration.RANSACConvergenceCriteria(4000000, 500))
            return result

        def refine_registration(source, target, voxel_size, trans_init):
            distance_threshold = voxel_size * 0.4
            result = o3d.registration.registration_icp(
                        source, target, distance_threshold, trans_init,
                        o3d.registration.TransformationEstimationPointToPlane())
            return result

        # get_icp_pose execution starts here
        source = crop_pcd(src)
        target = crop_pcd(dst)

        voxel_size = VOXEL_SZ
        source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(source, target, voxel_size)

        result_ransac = execute_global_registration(source_down, target_down,
                                                    source_fpfh, target_fpfh,
                                                    voxel_size)

        result_icp = refine_registration(source, target, voxel_size, result_ransac.transformation)


        evaluation = o3d.registration.evaluate_registration(source, target, voxel_size * 5, result_icp.transformation)
        return result_icp.transformation, evaluation
    
    def get_rpe(transform1, transform2, angle_err_wt=1):
        transformation_rpe =  np.matmul(np.linalg.inv(transform1), transform2)
        trans_arr, rot_mat, scale_mat, shear_mat = transforms3d.affines.decompose44(transformation_rpe)
        rot_list = transforms3d.euler.mat2euler(rot_mat, axes='sxyz')
        rot_arr = np.array(rot_list)
        rpe_total = np.linalg.norm(trans_arr) + (np.linalg.norm(rot_arr)*angle_err_wt)
        return rpe_total

    
    # icp_loss_fn starts here
    prev_pcd, next_pcd = transform_lidar_to_gt_frame(prev_pcd, next_pcd)

    transformation_gt = get_gt_pose(df_gt, idx)
    draw_registration_result(prev_pcd, next_pcd, transformation_gt)
    
    transformation_icp, evaluation_icp = get_icp_pose(prev_pcd, next_pcd)
    draw_registration_result(prev_pcd, next_pcd, transformation_icp)
    
    rpe_loss = get_rpe(transformation_gt, transformation_icp)
    
    return rpe_loss

In [30]:
class SupervisedConsecutivePairdata(torch.utils.data.Dataset):
    """
    Dataset of numbers in [a,b] inclusive
    """

    def __init__(self, dataset):
        super(SupervisedConsecutivePairdata, self).__init__()
        self.dataset_data  = dataset['data']
        self.dataset_label = dataset['label']
        
        assert len(self.dataset_data) == len(self.dataset_label)

    def __len__(self):
        return self.dataset_data.shape[0] - 1   # We don't want a pair for last frame

    def __getitem__(self, index):
        index1 = index
        index2 = index+1 if index+1 < self.dataset.shape[0] else index  # The pair for last lidar frame is itself
        
        return index, self.dataset[index1], self.dataset[index2], self.dataset_label.iloc[index1], self.dataset_label.iloc[index2]

In [ ]:
for file in tqdm_notebook(npyList):
    print(getint(file))

In [ ]:
train_data = ConsecutivePairdata(dynamic_dataset_train_list[0])
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64,
                    shuffle=False, num_workers=20, drop_last=False)

val_data = ConsecutivePairdata(dynamic_dataset_val)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=64,
                    shuffle=False, num_workers=20, drop_last=False)

In [ ]:
train_data = ConsecutivePairdata(dynamic_dataset_train)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64,
                    shuffle=False, num_workers=20, drop_last=False)

val_data = ConsecutivePairdata(dynamic_dataset_val)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=64,
                    shuffle=False, num_workers=20, drop_last=False)

In [ ]:
for i, img_data in tqdm_notebook(enumerate(train_loader), total=len(train_loader)):
    dynamic_img_curr = img_data[1].cuda()
    dynamic_img_next = img_data[2].cuda()
    break

Convert it back to pcd

In [ ]:
def get_pcd_from_img(img):
    frame = from_polar(img).detach().cpu().numpy()[0]
    frame_actual = np.array([frame_image[:29] for frame_image in frame])
    frame_flat = frame_actual.reshape((3,-1))
    some_pcd = o3d.PointCloud()
    some_arr = frame_flat.T
    some_pcd.points = o3d.utility.Vector3dVector(some_arr)
    return some_pcd

In [ ]:
dynamic_pcd_curr = get_pcd_from_img(dynamic_img_curr[0:1,:,:,:])
dynamic_pcd_next = get_pcd_from_img(dynamic_img_next[0:1,:,:,:])

In [ ]:
def slam_loss_fn(prev_pcd, next_pcd, idx, VOXEL_SZ=0.2):
    def transform_lidar_to_gt_frame(src, dst):
        source = copy.deepcopy(src)
        target = copy.deepcopy(dst)
        transformation_lidar2gt = pose2matrix([0,0,0], [0,0,90],[1,1,-1])
        source.transform(transformation_lidar2gt)
        target.transform(transformation_lidar2gt)
        return source, target
    
    def get_gt_pose(df_gt, idx):
        # get_gt_pose execution starts here
        prev_gt = df_gt[df_gt['step'] == idx +1].iloc[0]
        next_gt = df_gt[df_gt['step'] == idx+1 +1].iloc[0]

        prev_inv_mat = np.linalg.inv(pose2matrix([prev_gt['location_x'],
                                                  prev_gt['location_y'],
                                                  prev_gt['location_z']],
                                                 [prev_gt['rotation_roll'],
                                                  prev_gt['rotation_pitch'],
                                                  prev_gt['rotation_yaw']]))
        next_mat = pose2matrix([next_gt['location_x'],
                                next_gt['location_y'],
                                next_gt['location_z']],
                               [next_gt['rotation_roll'],
                                next_gt['rotation_pitch'],
                                next_gt['rotation_yaw']])
        transformation_gt = np.matmul(prev_inv_mat, next_mat)
        transformation_gt = np.linalg.inv(transformation_gt) # Open 3d assumes transform is applied on source and not target
        return transformation_gt
    
    def get_icp_pose(src, dst):
        def crop_pcd(old_pcd, crop_min_arr=np.array([-100,-100,-2]), crop_max_arr=np.array([100,100,100])):
            np.random.seed(0)
            pcd = copy.deepcopy(old_pcd)

            cropped_pcd = o3d.geometry.crop_point_cloud(pcd, crop_min_arr, crop_max_arr)
            pcd = cropped_pcd
            return pcd

        def prepare_dataset(source, target, voxel_size):
            def preprocess_point_cloud(pcd, voxel_size):
                pcd_down = o3d.geometry.voxel_down_sample(pcd, voxel_size)
                radius_normal = voxel_size * 2
                o3d.geometry.estimate_normals(
                    pcd_down,
                    o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
                radius_feature = voxel_size * 5
                pcd_fpfh = o3d.registration.compute_fpfh_feature(
                    pcd_down,
                    o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
                return pcd_down, pcd_fpfh
            source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
            target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
            return source_down, target_down, source_fpfh, target_fpfh

        def execute_global_registration(source_down, target_down, source_fpfh,
                                        target_fpfh, voxel_size):
            distance_threshold = voxel_size * 1.5
            result = o3d.registration.registration_ransac_based_on_feature_matching(
                source_down, target_down, source_fpfh, target_fpfh, distance_threshold,
                o3d.registration.TransformationEstimationPointToPoint(False), 4, [
                    o3d.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
                    o3d.registration.CorrespondenceCheckerBasedOnDistance(
                        distance_threshold)
                ], o3d.registration.RANSACConvergenceCriteria(4000000, 500))
            return result

        def refine_registration(source, target, voxel_size, trans_init):
            distance_threshold = voxel_size * 0.4
            result = o3d.registration.registration_icp(
                        source, target, distance_threshold, trans_init,
                        o3d.registration.TransformationEstimationPointToPlane())
            return result

        # get_icp_pose execution starts here
        source = crop_pcd(src)
        target = crop_pcd(dst)

        voxel_size = VOXEL_SZ
        source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(source, target, voxel_size)

        result_ransac = execute_global_registration(source_down, target_down,
                                                    source_fpfh, target_fpfh,
                                                    voxel_size)

        result_icp = refine_registration(source, target, voxel_size, result_ransac.transformation)


        evaluation = o3d.registration.evaluate_registration(source, target, voxel_size * 5, result_icp.transformation)
        return result_icp.transformation, evaluation
    
    def get_rpe(transform1, transform2, angle_err_wt=1):
        transformation_rpe =  np.matmul(np.linalg.inv(transform1), transform2)
        trans_arr, rot_mat, scale_mat, shear_mat = transforms3d.affines.decompose44(transformation_rpe)
        rot_list = transforms3d.euler.mat2euler(rot_mat, axes='sxyz')
        rot_arr = np.array(rot_list)
        rpe_total = np.linalg.norm(trans_arr) + (np.linalg.norm(rot_arr)*angle_err_wt)
        return rpe_total

    
    # icp_loss_fn starts here
    prev_pcd, next_pcd = transform_lidar_to_gt_frame(prev_pcd, next_pcd)

    transformation_gt = get_gt_pose(df_gt, idx)
    draw_registration_result(prev_pcd, next_pcd, transformation_gt)
    
    transformation_icp, evaluation_icp = get_icp_pose(prev_pcd, next_pcd)
    draw_registration_result(prev_pcd, next_pcd, transformation_icp)
    
    rpe_loss = get_rpe(transformation_gt, transformation_icp)
    
    return rpe_loss

In [ ]:
def draw_pcd(pcd, where='opn_nb'):    
    if where is 'opn_nb':
        visualizer = o3d.JVisualizer()
        visualizer.add_geometry(pcd)
        visualizer.show()
    elif where is 'opn_view':
        o3d.visualization.draw_geometries([pcd], width=1280, height=800)
    elif where is 'mat_3d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], pts[:,2])
        plt.show()
    elif where is 'mat_2d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], s=1)
        plt.show()

In [ ]:
draw_pcd(dynamic_pcd_curr)

In [3]:
some_dict = {}
some_dict['data'] = np.arange(10)
some_dict['label'] = np.zeros(10)

In [5]:
some_dict

{'data': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 'label': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])}

In [8]:
def dict_idx(that_dict, idx):
    return {k:v[6] for k, v in that_dict.items()}

{'data': 6, 'label': 0.0}

In [16]:
x  = np.array([[10]])

In [17]:
x *= 6

In [18]:
x

array([[60]])

In [19]:
type(x)

numpy.ndarray

In [28]:
torch.from_numpy(np.array(10))

tensor(10)

In [22]:
torch.Tensor(10)

tensor([1.3058e+29, 3.0707e-41, 8.4078e-45, 0.0000e+00,        nan, 0.0000e+00,
        1.8617e+25, 4.0690e+31, 5.8855e-44, 0.0000e+00])

In [26]:
torch.from_numpy?

In [29]:
Variable(torch.from_numpy(np.array(10)), )

tensor(10)